In [7]:
# Bước 1: Cài đặt thư viện cần thiết (nếu chưa có)
#!pip install gspread pandas google-auth-oauthlib google-auth-httplib2

In [8]:
# Bước 2: Import thư viện
import pandas as pd
import requests
import io
from google.colab import auth
from google.colab import files # Để tải file kết quả về máy
from google.auth import default
import os # Để kiểm tra file tồn tại

In [9]:
# --- Bước 3: Xác thực người dùng ---
# Vẫn cần thiết để có quyền tải file Google Sheet nếu nó không công khai hoàn toàn
print("Authenticating user...")
try:
    auth.authenticate_user()
    creds, _ = default()
    print("Authentication successful (may be needed for download).")
except Exception as e:
    print(f"Authentication failed: {e}")
    # Thoát nếu xác thực không thành công
    exit()

Authenticating user...
Authentication successful (may be needed for download).


In [10]:
# --- Bước 4: Tải file Google Sheet về dưới dạng Excel (.xlsx) ---
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1uSUkePcSsuECugJMdkzxw4RLpD9xAphXa6vVObSP2oA/edit?usp=sharing'
# Trích xuất File ID từ URL
try:
    file_id = google_sheet_url.split('/d/')[1].split('/')[0]
except IndexError:
    print("Error: Could not extract File ID from the Google Sheet URL.")
    exit()

# Tạo URL export sang định dạng xlsx
download_url = f'https://docs.google.com/spreadsheets/d/{file_id}/export?format=xlsx'
local_xlsx_filename = 'downloaded_sheet.xlsx' # Tên file sẽ lưu trong Colab

print(f"\nAttempting to download Google Sheet as '{local_xlsx_filename}'...")
print(f"Export URL: {download_url}")

try:
    # Chuẩn bị header xác thực cho requests
    import google.auth.transport.requests
    request_session = google.auth.transport.requests.Request()
    creds.refresh(request_session) # Làm mới token nếu cần
    headers = {'Authorization': 'Bearer ' + creds.token}

    # Thực hiện yêu cầu tải file
    response = requests.get(download_url, headers=headers, stream=True) # stream=True tốt cho file lớn
    response.raise_for_status() # Báo lỗi nếu có lỗi HTTP (4xx, 5xx)

    # Ghi nội dung tải về vào file cục bộ trong Colab
    with open(local_xlsx_filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Successfully downloaded and saved as '{local_xlsx_filename}'.")

except requests.exceptions.RequestException as e:
    print(f"Error downloading file: {e}")
    print("Possible reasons:")
    print("- Incorrect Google Sheet URL or File ID.")
    print("- Insufficient permissions for the authenticated user to access/download.")
    print("- Network issues.")
    exit()
except Exception as e:
    print(f"An unexpected error occurred during download: {e}")
    exit()



Attempting to download Google Sheet as 'downloaded_sheet.xlsx'...
Export URL: https://docs.google.com/spreadsheets/d/1uSUkePcSsuECugJMdkzxw4RLpD9xAphXa6vVObSP2oA/export?format=xlsx
Successfully downloaded and saved as 'downloaded_sheet.xlsx'.


In [11]:

# --- Bước 5: Xử lý file Excel đã tải về ---
sheet_names_to_process = ['FAQ', 'SP', 'EVEN']
all_data = [] # List để lưu các DataFrame đã xử lý
print(f"\nProcessing sheets from '{local_xlsx_filename}': {', '.join(sheet_names_to_process)}")

# Kiểm tra file đã tải về có tồn tại không
if not os.path.exists(local_xlsx_filename):
     print(f"Error: The downloaded file '{local_xlsx_filename}' was not found. Cannot process.")
     exit()

try:
    # Đọc các sheet cụ thể từ file Excel vào một dictionary
    # header=0 nghĩa là dòng đầu tiên trong Excel là dòng tiêu đề
    excel_data = pd.read_excel(local_xlsx_filename,
                               sheet_name=sheet_names_to_process,
                               header=0,
                               engine='openpyxl') # Chỉ định engine để đọc xlsx

    # Lặp qua dictionary các DataFrame (mỗi key là tên sheet)
    for sheet_name, df_sheet in excel_data.items():
        print(f" - Processing sheet: '{sheet_name}'...")
        # Kiểm tra xem sheet có dữ liệu và đủ 2 cột không
        if not df_sheet.empty and len(df_sheet.columns) >= 2:
            # Chọn 2 cột đầu tiên (giả định là Question và Answer)
            # iloc[:, [0, 1]] chọn cột theo vị trí index
            df_processed = df_sheet.iloc[:, [0, 1]].copy()
            # Đặt lại tên cột
            df_processed.columns = ['Question', 'Answer']
            # Thêm cột Category
            df_processed['Category'] = sheet_name

            # Loại bỏ các hàng có 'Question' là NaN hoặc rỗng/chỉ chứa khoảng trắng
            df_processed.dropna(subset=['Question'], inplace=True)
            # Chuyển sang string để dùng .str.strip()
            df_processed = df_processed[df_processed['Question'].astype(str).str.strip() != '']

            # Chỉ thêm vào list nếu có dữ liệu hợp lệ sau khi xử lý
            if not df_processed.empty:
                 all_data.append(df_processed)
                 print(f"   > Found and processed {len(df_processed)} valid rows.")
            else:
                 print(f"   > No valid data rows found after processing.")

        elif df_sheet.empty:
             print(f"   > Warning: Sheet '{sheet_name}' is empty. Skipping.")
        else: # Không đủ 2 cột
            print(f"   > Warning: Sheet '{sheet_name}' has fewer than 2 columns. Skipping.")

except ValueError as e:
     # Lỗi này thường xảy ra nếu tên sheet không tồn tại trong file Excel
     print(f"Error reading Excel file: {e}")
     print(f"Please ensure the sheet names ({', '.join(sheet_names_to_process)}) exist in '{local_xlsx_filename}'.")
     exit()
except Exception as e:
    print(f"An unexpected error occurred while processing the Excel file: {e}")
    exit()



Processing sheets from 'downloaded_sheet.xlsx': FAQ, SP, EVEN
 - Processing sheet: 'FAQ'...
   > Found and processed 12 valid rows.
 - Processing sheet: 'SP'...
   > Found and processed 10 valid rows.
 - Processing sheet: 'EVEN'...
   > Found and processed 9 valid rows.


In [12]:

# --- Bước 6: Kết hợp dữ liệu và Lưu file CSV ---
if all_data:
    print("\nCombining data from processed sheets...")
    df_combined = pd.concat(all_data, ignore_index=True)
    # Đảm bảo đúng thứ tự cột
    df_combined = df_combined[['Category', 'Question', 'Answer']]
    print("Data combined successfully.")

    output_filename = 'knowledge_base.csv'
    print(f"Saving combined data to '{output_filename}'...")
    try:
        # Lưu ra file CSV trong môi trường Colab
        df_combined.to_csv(output_filename, index=False, encoding='utf-8')
        print(f"Successfully saved {len(df_combined)} rows to '{output_filename}'.")

        # Hiển thị 5 dòng đầu tiên để kiểm tra (tùy chọn)
        print("\nFirst 5 rows of the combined data:")
        print(df_combined.head().to_markdown(index=False, numalign="left", stralign="left"))

        # --- Bước 7: Tải file CSV kết quả về máy tính ---
        print(f"\nPreparing to download '{output_filename}'...")
        if os.path.exists(output_filename):
            try:
                files.download(output_filename) # Kích hoạt tải file về trình duyệt
                print(f"Download initiated for '{output_filename}'. Please check your browser.")
            except Exception as e:
                print(f"An error occurred during CSV download: {e}")
        else:
            print(f"Error: Output file '{output_filename}' not found for download.")

    except Exception as e:
        print(f"Error saving file '{output_filename}': {e}")

else:
    # Trường hợp không có dữ liệu nào được xử lý thành công
    print("\nNo data was successfully processed from any sheet. Output file 'knowledge_base.csv' was not created.")


Combining data from processed sheets...
Data combined successfully.
Saving combined data to 'knowledge_base.csv'...
Successfully saved 31 rows to 'knowledge_base.csv'.

First 5 rows of the combined data:
| Category   | Question                                       | Answer                                                                                                                                          |
|:-----------|:-----------------------------------------------|:------------------------------------------------------------------------------------------------------------------------------------------------|
| FAQ        | Địa chỉ trường mình ở đâu?                     | Trường [Tên trường] có địa chỉ tại [Địa chỉ cụ thể]. Bạn có thể xem bản đồ chi tiết trên website [Link bản đồ nếu có].                          |
| FAQ        | Số điện thoại liên hệ của trường là gì?        | Bạn có thể liên hệ phòng ban chung qua số [Số điện thoại chung] hoặc truy cập [Link trang liên hệ] để

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download initiated for 'knowledge_base.csv'. Please check your browser.
